In [7]:
import json
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D, Dense, Embedding, Flatten
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import random

# Load intents data
with open("data.json", "r") as file:
    data = json.load(file)

# Data preprocessing
patterns = []
labels = []
tag_classes = []

tag_to_response = {}

for intent in data["intents"]:
    tag = intent["tag"]
    tag_classes.append(tag)
    tag_to_response[tag] = intent["responses"]
    for pattern in intent["patterns"]:
        patterns.append(pattern)
        labels.append(tag)

# Tokenize the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(patterns)
word_index = tokenizer.word_index
X_sequences = tokenizer.texts_to_sequences(patterns)
X_padded = pad_sequences(X_sequences, padding="post")

# Convert labels to numeric values
tag_dict = {tag: i for i, tag in enumerate(tag_classes)}
y_train = np.array([tag_dict[label] for label in labels])

# Build CNN model
model = Sequential([
    Embedding(input_dim=len(word_index) + 1, output_dim=64, input_length=X_padded.shape[1]),
    Conv1D(64, 5, activation='relu'),
    GlobalMaxPooling1D(),
    Dense(32, activation='relu'),
    Dense(len(tag_classes), activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_padded, y_train, epochs=100, verbose=1)

# Function to predict response
def chatbot_response(user_input):
    input_seq = tokenizer.texts_to_sequences([user_input])
    input_padded = pad_sequences(input_seq, maxlen=X_padded.shape[1], padding='post')
    prediction = model.predict(input_padded)
    tag_index = np.argmax(prediction)
    tag = tag_classes[tag_index]
    return random.choice(tag_to_response[tag])

# Chatbot interaction loop
while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit", "bye"]:
        print("Chatbot: Goodbye!")
        break
    response = chatbot_response(user_input)
    print("Chatbot:", response)


Epoch 1/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - accuracy: 0.0918 - loss: 2.3081
Epoch 2/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.2148 - loss: 2.2910
Epoch 3/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.3557 - loss: 2.2775
Epoch 4/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.5175 - loss: 2.2661
Epoch 5/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.5705 - loss: 2.2544
Epoch 6/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.5951 - loss: 2.2435
Epoch 7/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.6055 - loss: 2.2301
Epoch 8/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.6159 - loss: 2.2137
Epoch 9/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.5988 - loss: 2.1982
Epoch 10/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.5884 - loss: 2.1850
Epoch 11/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.5988 - loss: 2.1670
Epoch 12/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.6197 - lo